In [ ]:
!pip install split-folders

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile
import tensorflow as tf
import os
from matplotlib import pyplot as plt
import cv2
from google.colab import files
import numpy as np
from PIL import Image
import splitfolders
import time

#from tensorflow.keras.applications import DenseNet201
#from tensorflow.keras.applications.densenet import preprocess_input as proses_densenet

#from tensorflow.keras.applications import NASNetMobile
#from tensorflow.keras.applications import NASNetLarge
#from tensorflow.keras.applications.nasnet import preprocess_input as proses_nasnet

from tensorflow.keras.applications import EfficientNetB2
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras.applications import EfficientNetB6
from tensorflow.keras.applications.efficientnet import preprocess_input as proses_efficientnet

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers.experimental import Adamax
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report

In [ ]:
input_folder = '/content/drive/MyDrive/Penelitian/Dataset/Lymphoma'
output = '/content/drive/MyDrive/Penelitian/Dataset/Lymphoma_new'
splitfolders.ratio(input_folder, output, seed=42, ratio=(.7, .15, .15))

Copying files: 374 files [00:26, 13.94 files/s]


# Load Gambar

In [ ]:
img_height, img_width = 1388, 1040
dimension = (img_height, img_width)
batch_size = 32

train_data_dir = r"/content/drive/MyDrive/Penelitian/Dataset/Lymphoma_new/train"
valid_data_dir = r"/content/drive/MyDrive/Penelitian/Dataset/Lymphoma_new/val"
test_data_dir = r"/content/drive/MyDrive/Penelitian/Dataset/Lymphoma_new/test"

# EFFICIENT-NETB2

In [ ]:
traindatagen = ImageDataGenerator(
    #rescale = 1.0/255,
    preprocessing_function=proses_efficientnet,
)
train_data = traindatagen.flow_from_directory(
    directory = train_data_dir,
    target_size = dimension
)

validdatagen = ImageDataGenerator(
    #rescale = 1.0/255,
    preprocessing_function=proses_efficientnet,
)
valid_data = validdatagen.flow_from_directory(
    directory = valid_data_dir,
    target_size = dimension
)

testdatagen = ImageDataGenerator(
    #rescale = 1.0/255,
    preprocessing_function=proses_efficientnet,
)
test_data = testdatagen.flow_from_directory(
    directory = test_data_dir,
    target_size = dimension
)

Found 262 images belonging to 3 classes.
Found 55 images belonging to 3 classes.
Found 61 images belonging to 3 classes.


In [ ]:
start_time = time.time()

In [ ]:
base_model = EfficientNetB2(include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_data.num_classes, activation='softmax')(x)
model = Model(inputs = base_model.input, outputs=predictions)

for layer in base_model.layers:
  layer.trainable = False

model.compile(loss='categorical_crossentropy', metrics = ['accuracy'])
model.fit(train_data, epochs=50,validation_data=valid_data, verbose = 1)

Epoch 1/50
9/9 [==============================] - 91s 8s/step - loss: 2.8495 - accuracy: 0.3969 - val_loss: 0.9315 - val_accuracy: 0.5636
Epoch 2/50
9/9 [==============================] - 30s 3s/step - loss: 0.8948 - accuracy: 0.5916 - val_loss: 1.3286 - val_accuracy: 0.4545
Epoch 3/50
9/9 [==============================] - 30s 3s/step - loss: 1.0600 - accuracy: 0.5611 - val_loss: 0.7347 - val_accuracy: 0.6182
Epoch 4/50
9/9 [==============================] - 32s 3s/step - loss: 0.8067 - accuracy: 0.6527 - val_loss: 0.8686 - val_accuracy: 0.6364
Epoch 5/50
9/9 [==============================] - 31s 3s/step - loss: 0.8705 - accuracy: 0.5687 - val_loss: 0.8496 - val_accuracy: 0.6000
Epoch 6/50
9/9 [==============================] - 30s 3s/step - loss: 0.8107 - accuracy: 0.6221 - val_loss: 1.0846 - val_accuracy: 0.6182
Epoch 7/50
9/9 [==============================] - 30s 3s/step - loss: 0.8667 - accuracy: 0.6069 - val_loss: 0.6601 - val_accuracy: 0.7818
Epoch 8/50
9/9 [==================

In [ ]:
end_time = time.time()
execution_time = end_time - start_time
print("Lama eksekusi:", execution_time, "detik")

Lama eksekusi: 1708.8615539073944 detik


In [ ]:
test_loss, test_acc = model.evaluate(test_data, verbose=1)
print('\nTest accuracy: ', test_acc)

2/2 [==============================] - 16s 12s/step - loss: 0.3032 - accuracy: 0.8361

Test accuracy:  0.8360655903816223


In [ ]:
y_pred = model.predict(test_data)
y_true = test_data.classes
y_pred = np.argmax(y_pred, axis=1)

classification_rep = classification_report(y_true, y_pred, target_names=test_data.class_indices)
print(classification_rep)

2/2 [==============================] - 7s 3s/step
              precision    recall  f1-score   support

         CLL       0.35      0.44      0.39        18
          FL       0.39      0.41      0.40        22
         MCL       0.47      0.33      0.39        21

    accuracy                           0.39        61
   macro avg       0.40      0.40      0.39        61
weighted avg       0.40      0.39      0.39        61



In [ ]:
start_time2 = time.time()

In [ ]:
base_model = EfficientNetB2(include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_data.num_classes, activation='softmax')(x)
model = Model(inputs = base_model.input, outputs=predictions)

for layer in base_model.layers:
  layer.trainable = False

model.compile(loss='categorical_crossentropy', metrics = ['accuracy'])
model.fit(train_data, epochs=50,validation_data=valid_data, verbose = 1)

Epoch 1/50
9/9 [==============================] - 44s 4s/step - loss: 2.1190 - accuracy: 0.3779 - val_loss: 0.8866 - val_accuracy: 0.4909
Epoch 2/50
9/9 [==============================] - 30s 3s/step - loss: 1.0361 - accuracy: 0.5573 - val_loss: 0.7936 - val_accuracy: 0.5636
Epoch 3/50
9/9 [==============================] - 30s 3s/step - loss: 0.9182 - accuracy: 0.5267 - val_loss: 0.9136 - val_accuracy: 0.5636
Epoch 4/50
9/9 [==============================] - 35s 4s/step - loss: 0.9234 - accuracy: 0.5802 - val_loss: 0.8044 - val_accuracy: 0.6364
Epoch 5/50
9/9 [==============================] - 30s 3s/step - loss: 0.7886 - accuracy: 0.6450 - val_loss: 0.8372 - val_accuracy: 0.6182
Epoch 6/50
9/9 [==============================] - 35s 4s/step - loss: 0.8553 - accuracy: 0.5916 - val_loss: 0.6259 - val_accuracy: 0.7455
Epoch 7/50
9/9 [==============================] - 31s 3s/step - loss: 0.7304 - accuracy: 0.7366 - val_loss: 0.8301 - val_accuracy: 0.5273
Epoch 8/50
9/9 [==================

In [ ]:
end_time2 = time.time()
execution_time2 = end_time2 - start_time2
print("Lama eksekusi kedua:", execution_time2, "detik")

Lama eksekusi kedua: 1669.3908097743988 detik


In [ ]:
test_loss, test_acc = model.evaluate(test_data, verbose=1)
print('\nTest accuracy: ', test_acc)

2/2 [==============================] - 6s 2s/step - loss: 0.2600 - accuracy: 0.9016

Test accuracy:  0.9016393423080444


In [ ]:
y_pred = model.predict(test_data)
y_true = test_data.classes
y_pred = np.argmax(y_pred, axis=1)

classification_rep = classification_report(y_true, y_pred, target_names=test_data.class_indices)
print(classification_rep)

2/2 [==============================] - 7s 2s/step
              precision    recall  f1-score   support

         CLL       0.35      0.33      0.34        18
          FL       0.42      0.45      0.43        22
         MCL       0.40      0.38      0.39        21

    accuracy                           0.39        61
   macro avg       0.39      0.39      0.39        61
weighted avg       0.39      0.39      0.39        61



In [ ]:
print(y_true)
print(y_pred)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[0 2 2 2 1 0 0 1 0 0 2 1 2 1 1 2 0 1 1 2 2 0 2 1 0 1 1 2 2 0 1 0 0 1 1 1 1
 0 1 2 0 2 1 1 2 2 1 1 0 1 2 2 2 1 1 2 0 0 2 0 1]
